In [1]:
# Cellule 1 – Imports et chemins

# ============================================================
# Notebook 07 : Création des masques pour test_eliva25
# ============================================================

import os
from pathlib import Path

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing.image import array_to_img

print("TensorFlow version :", tf.__version__)

BASE_DIR = Path(".").resolve()
print("BASE_DIR :", BASE_DIR)

# Dossier contenant les images ELIVA25 (images test)
ELIVA_DIR = BASE_DIR / "test_eliva25"

# Dossier de sortie
OUT_DIR = BASE_DIR / "pred_masks_eliva25"
OUT_DIR.mkdir(exist_ok=True)

print("Images Eliva25 :", ELIVA_DIR)
print("Dossier sortie :", OUT_DIR)

# Vérif
if not ELIVA_DIR.exists():
    raise FileNotFoundError("Le dossier test_eliva25 est introuvable.")


2025-12-01 17:22:48.160513: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-01 17:22:48.173711: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-01 17:22:48.187619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-01 17:22:48.191935: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-01 17:22:48.203391: I tensorflow/core/platform/cpu_feature_guar

TensorFlow version : 2.17.0
BASE_DIR : /home/fabrice/Boneage
Images Eliva25 : /home/fabrice/Boneage/test_eliva25
Dossier sortie : /home/fabrice/Boneage/pred_masks_eliva25


In [2]:
# Cellule 2 – Chargement du modèle entraîné

# ============================================================
# Chargement modèle segmentation
# ============================================================

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    inter = tf.reduce_sum(y_true * y_pred)
    return (2*inter + smooth) / (
        tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth
    )

def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return bce + (1 - dice_coef(y_true, y_pred))

model = keras.models.load_model(
    "seg_model_best.keras",
    custom_objects={"dice_coef": dice_coef, "bce_dice_loss": bce_dice_loss}
)

print("Modèle chargé.")
model.summary()


I0000 00:00:1764609824.012772   10579 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764609824.202648   10579 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764609824.202733   10579 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764609824.205125   10579 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764609824.205229   10579 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Modèle chargé.


Model: "ResNet50_UNet_seg"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 224, 224)  │          0 │ image_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 224, 224)  │          0 │ image_input[0][0] │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 224, 224)  │          0 │ image_input[0][0] │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack (Stack)       │ (None, 224, 224,  │          0 │ get_item[0][0],   │
│                     │ 3)                │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 224, 224,  │          0 │ stack[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ add[0][0]         │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c

 Total params: 131,815,461 (502.84 MB)

 Trainable params: 43,918,881 (167.54 MB)

 Non-trainable params: 58,816 (229.75 KB)

 Optimizer params: 87,837,764 (335.07 MB)

In [3]:
# Cellule 3 – Fonction de preprocessing (identique à l’entraînement)

# ============================================================
# Préprocessing identique à l'entraînement
# ============================================================

IMG_SIZE = 224

def preprocess_image_for_model(path):
    """Charge une image Eliva25 en reproduisant EXACTEMENT le preprocessing du training."""
    
    img_bytes = tf.io.read_file(path)
    img = tf.image.decode_png(img_bytes, channels=1)   # 1 canal
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])   # 224x224
    
    img = tf.image.grayscale_to_rgb(img)               # → (224,224,3)
    img = tf.cast(img, tf.float32)                     # IMPORTANT : pas de normalisation ici
    
    return img


In [4]:
# Cellule 4 – Liste des images ELIVA25

# ============================================================
# Récupération liste d'images du dossier test_eliva25
# ============================================================

png_files = sorted([f for f in ELIVA_DIR.iterdir() if f.suffix.lower() == ".png"])

print(f"Nombre d'images ELIVA25 détectées : {len(png_files)}")
if len(png_files) == 0:
    raise ValueError("Aucune image PNG trouvée dans test_eliva25.")


Nombre d'images ELIVA25 détectées : 33


In [7]:
# Cellule 5 – Fonction de prédiction de masque

# ============================================================
# Prédiction d'un masque à partir d'un tenseur image (224x224x3)
# ============================================================

def predict_mask(image_tensor):
    """
    image_tensor : tf.Tensor ou array (224,224,3), float32 en [0..255]
    Retourne : masque binaire numpy (224,224), valeurs 0 ou 1 (uint8)
    """
    # Sécurisation du type
    if not isinstance(image_tensor, tf.Tensor):
        image_tensor = tf.convert_to_tensor(image_tensor, dtype=tf.float32)
    else:
        image_tensor = tf.cast(image_tensor, tf.float32)

    # Ajout de la dimension batch
    img_batch = tf.expand_dims(image_tensor, axis=0)  # (1,224,224,3)

    # Prédiction modèle (sortie : (1,224,224,1), ndarray)
    pred = model.predict(img_batch, verbose=0)
    pred = pred[0, ..., 0]  # (224,224), numpy

    # Seuillage binaire
    pred_mask = (pred > 0.5).astype(np.uint8)  # 0/1
    return pred_mask


In [9]:
# Cellule 6 – Génération & sauvegarde des masques

from tqdm import tqdm
import numpy as np

num_images = len(png_files)
print(f"Nombre d'images à traiter : {num_images}")

for path in tqdm(png_files, desc="Prédiction des masques"):
    # 1) Preprocessing identique au training
    img = preprocess_image_for_model(str(path))   # tf.Tensor (224,224,3)

    # 2) Prédiction du masque binaire
    pred_mask = predict_mask(img)                 # numpy (224,224), 0/1

    # 3) Conversion en image 0–255 pour sauvegarde PNG
    pred_mask_img = array_to_img(
        np.expand_dims(pred_mask * 255, axis=-1)  # (224,224,1)
    )

    # 4) Sauvegarde
    out_path = OUT_DIR / path.name
    pred_mask_img.save(out_path)

print("Génération des masques terminée.")


Nombre d'images à traiter : 33


Prédiction des masques: 100%|███████████████████████████████████████████████████████████| 33/33 [00:02<00:00, 12.77it/s]

Génération des masques terminée.


In [10]:
# Cellule 7 — Vérification binaire sur tous les masques

import numpy as np
from PIL import Image

errors = []

for mask_path in sorted(OUT_DIR.iterdir()):
    if mask_path.suffix.lower() != ".png":
        continue
    
    mask = np.array(Image.open(mask_path))

    # Vérification : uniquement 0 ou 255 ?
    unique_vals = np.unique(mask)

    if not np.array_equal(unique_vals, [0]) and \
       not np.array_equal(unique_vals, [255]) and \
       not np.array_equal(unique_vals, [0, 255]):
        errors.append((mask_path.name, unique_vals))

if errors:
    print(" Masques NON binaires détectés :")
    for name, vals in errors:
        print(f"{name}: valeurs uniques = {vals}")
else:
    print(" Tous les masques générés sont strictement binaires (0 et/ou 255).")


 Tous les masques générés sont strictement binaires (0 et/ou 255).


In [11]:
# Cellule 8 — Vérification des masques binaires dans eff_unet

from pathlib import Path
import numpy as np
from PIL import Image

MASKS_DIR = Path("eff_unet")

if not MASKS_DIR.exists():
    raise FileNotFoundError(f"Dossier introuvable : {MASKS_DIR}")

non_binary_masks = []
all_masks = sorted([f for f in MASKS_DIR.iterdir() if f.suffix.lower() == ".png"])

print(f"Nombre de masques trouvés : {len(all_masks)}")

for mask_path in all_masks:
    mask = np.array(Image.open(mask_path))

    # récupération des valeurs uniques
    unique_vals = np.unique(mask)

    # condition : uniquement 0 ou 255
    if not np.array_equal(unique_vals, [0]) and \
       not np.array_equal(unique_vals, [255]) and \
       not np.array_equal(unique_vals, [0, 255]):
        non_binary_masks.append((mask_path.name, unique_vals))

if len(non_binary_masks) == 0:
    print(" Tous les masques du dossier eff_unet sont strictement binaires (0 et/ou 255).")
else:
    print(" Masques non binaires détectés :")
    for name, vals in non_binary_masks:
        print(f"  - {name} : valeurs uniques = {vals}")


Nombre de masques trouvés : 14213
 Tous les masques du dossier eff_unet sont strictement binaires (0 et/ou 255).
